在求梯度的过程中使用向量化运算代替for循环

$ \dfrac {2}{m} · \begin{bmatrix} \sum ^{m}_{i=1}\left( x^{(i)}_b\theta -y^{(i)}\right) \\  \sum ^{m}_{i=1}\left( x^{(i)}_b\theta -y^{(i)}\right)·x^{(i)}_1 \\  \sum ^{m}_{i=1}\left( x^{(i)}_b\theta -y^{(i)}\right)·x^{(i)}_2 \\ ... \\  \sum ^{m}_{i=1}\left( x^{(i)}_b\theta -y^{(i)}\right)·x^{(i)}_n \end{bmatrix} =  \dfrac {2}{m} · \begin{bmatrix} \sum ^{m}_{i=1}\left( x^{(i)}_b\theta -y^{(i)}\right)·x^{(i)}_0 \\  \sum ^{m}_{i=1}\left( x^{(i)}_b\theta -y^{(i)}\right)·x^{(i)}_1 \\  \sum ^{m}_{i=1}\left( x^{(i)}_b\theta -y^{(i)}\right)·x^{(i)}_2 \\ ... \\  \sum ^{m}_{i=1}\left( x^{(i)}_b\theta -y^{(i)}\right)·x^{(i)}_n \end{bmatrix}$

每一组先乘后连加，都对应矩阵运算中，一行和一列的乘积。将这种公式变形成矩阵形式时，可以先对一个先乘后加的组合进行拆解，然后各自扩展成一行和一列的形式，然后慢慢寻找规律。之后再调整成为符合矩阵乘法法则的形式。比如这里，i从1取到m，则对前面$\left( x^{(i)}_b\theta -y^{(i)}\right)$的部分，扩展成$\left( x^{(1)}_b\theta -y^{(1)}, x^{(2)}_b\theta -y^{(2)}, x^{(3)}_b\theta -y^{(3)},..., x^{(m)}_b\theta -y^{(m)}\right)$.对后面的$ x^{(i)}_0$,则扩展成$(x^{(1)}_0,x^{(2)}_0,x^{(3)}_0,...,x^{(m)}_0)$  
以此类推，扩展全部的式子，之后再各自整合成矩阵的形式，然后再调整先后顺序，以满足运算法则。

原式 = $\dfrac {2}{m} \left( x^{(1)}_b\theta -y^{(1)}, x^{(2)}_b\theta -y^{(2)}, x^{(3)}_b\theta -y^{(3)},..., x^{(m)}_b\theta -y^{(m)}\right)·\begin{bmatrix} x^{(1)}_{0}  & x^{(1)}_{1} & x^{(1)}_{2} & ... & x^{(1)}_{n} \\ x^{(2)}_{0} & x^{(2)}_{1} & x^{(2)}_{2} & ... & x^{(2)}_{n}\\...\\ x^{(m)}_{0} & x^{(m)}_{1} & x^{(m)}_{2} & ... & x^{(m)}_{n} \end{bmatrix} = \dfrac {2}{m} (X_b \theta - y)^T · X_b$  
虽然numpy中对于向量，不区分是列向量还是行向量，但是基于默认向量都是列向量的共识，行向量还是需要在公式里标清楚  
所以最终结果需要写成：$\dfrac {2}{m}X_b^T · (X_b \theta - y) $

### 梯度下降向量化运算

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
boston = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y<50.0]
y = y[y<50.0]

In [3]:
from playML.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

In [4]:
from playML.LinearRegression import LinearRegression

lin_reg1 = LinearRegression()
%time lin_reg1.fit_normal(X_train, y_train)
lin_reg1.score(X_test, y_test)

CPU times: user 5.49 ms, sys: 1.61 ms, total: 7.09 ms
Wall time: 5.88 ms


0.9582380449074678

In [5]:
lin_reg2 = LinearRegression()
lin_reg2.fit_gd(X_train, y_train)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/apple/learnspace/机器学习/playML/LinearRegression.py:36: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
/Users/apple/learnspace/机器学习/playML/LinearRegression.py:58: RuntimeWarning: invalid value encountered in double_scalars
  if(abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):


LinearRegression()

In [6]:
lin_reg2.coef_

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [7]:
X_train[:10, :]

array([[1.42362e+01, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.93000e-01,
        6.34300e+00, 1.00000e+02, 1.57410e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.96900e+02, 2.03200e+01],
       [3.67822e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 7.70000e-01,
        5.36200e+00, 9.62000e+01, 2.10360e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.80790e+02, 1.01900e+01],
       [1.04690e-01, 4.00000e+01, 6.41000e+00, 1.00000e+00, 4.47000e-01,
        7.26700e+00, 4.90000e+01, 4.78720e+00, 4.00000e+00, 2.54000e+02,
        1.76000e+01, 3.89250e+02, 6.05000e+00],
       [1.15172e+00, 0.00000e+00, 8.14000e+00, 0.00000e+00, 5.38000e-01,
        5.70100e+00, 9.50000e+01, 3.78720e+00, 4.00000e+00, 3.07000e+02,
        2.10000e+01, 3.58770e+02, 1.83500e+01],
       [6.58800e-02, 0.00000e+00, 2.46000e+00, 0.00000e+00, 4.88000e-01,
        7.76500e+00, 8.33000e+01, 2.74100e+00, 3.00000e+00, 1.93000e+02,
        1.78000e+01, 3.95560e+02, 7.56000e+00],
       [2.49800e-02, 0.00000e+

这种情况说明梯度可能还是有点大，不能收敛

In [8]:
lin_reg2.fit_gd(X_train, y_train, eta=1e-6)

LinearRegression()

In [9]:
lin_reg2.score(X_test, y_test)

0.9167936632689727

现在不报错了，但是结果不如正规解，可能是因为步长太小，走了n_iters次，还没收敛到极小值上，此时可以增大n_iters，增加寻找次数

In [10]:
lin_reg2.fit_gd(X_train, y_train, eta=1e-6, n_iters=1e6)

LinearRegression()

In [11]:
lin_reg2.score(X_test, y_test)

0.9167936632689727

在视频里，修改之后是有提高的，但是还是不如正规解。不知道为什么我这里就没有再提分了。。。。

### 梯度下降与数据归一化
数据归一化后，更利于寻找极小值。所以梯度下降法前，最好进行数据归一化

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
standardScaler = StandardScaler()
standardScaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
X_train_standard = standardScaler.transform(X_train)

In [15]:
lin_reg3 = LinearRegression()
%time lin_reg3.fit_gd(X_train_standard, y_train)

CPU times: user 1.66 s, sys: 33.6 ms, total: 1.7 s
Wall time: 432 ms


LinearRegression()

In [16]:
X_test_standard = standardScaler.transform(X_test)

In [17]:
lin_reg3.score(X_test_standard, y_test)

0.9582418214741696

可以观察到，使用归一化后，即便在默认的eta和n_iters下，也能找到和正规解一样的极小值

### 梯度下降的优势

In [18]:
#设计虚拟测试用例
m = 1000
n = 5000
#随机生成X
big_X = np.random.normal(size = (m, n))
#生成对应的y,由于要生成的是线性模型，先生成对应的theta
true_theta = np.random.uniform(0.0, 100.0, size = n + 1)
#噪音：np.random.normal(0., 10., size = m)

In [19]:
big_y = big_X.dot(true_theta[1:]) + true_theta[0] + np.random.normal(0., 10., size = m)

In [20]:
big_y

array([-5.36583854e+03,  1.78249373e+03,  2.98409073e+03,  3.66377778e+03,
       -7.72885382e+02, -3.16396786e+03, -2.86655589e+00, -8.09465311e+02,
        1.99711472e+03, -2.47499104e+03,  2.02273540e+03, -3.18845023e+03,
       -2.30295639e+03, -3.88451688e+02,  4.30088321e+03, -2.87218928e+03,
        8.65983893e+03,  2.15781253e+03,  4.26588496e+03,  9.60131435e+03,
        5.59244160e+02,  4.28977476e+03,  4.10204539e+03,  2.59073257e+03,
        1.60453368e+03,  4.82030364e+03, -2.46537155e+03,  2.27343616e+03,
        4.37575345e+03, -8.46577441e+02,  1.82917474e+02, -3.38671498e+03,
        4.35181849e+02, -2.05620394e+03,  1.37935034e+03, -2.32873679e+03,
        1.27191928e+03,  1.08918549e+03, -5.09016348e+02,  3.34399482e+03,
       -2.57739144e+02,  3.95966407e+02, -2.55076866e+03, -4.06728924e+03,
       -1.56994046e+03, -1.86526627e+03, -3.38855470e+03, -1.68750773e+03,
       -3.40793609e+03,  5.76615451e+03, -4.58128238e+03,  7.11968638e+03,
       -3.03176487e+03, -

In [21]:
big_reg1 = LinearRegression()
%time big_reg1.fit_normal(big_X, big_y)

CPU times: user 31.8 s, sys: 794 ms, total: 32.6 s
Wall time: 8.36 s


LinearRegression()

In [22]:
big_reg2 = LinearRegression()
%time big_reg2.fit_gd(big_X, big_y)

CPU times: user 13.6 s, sys: 246 ms, total: 13.8 s
Wall time: 3.71 s


LinearRegression()

数据的维度较多的时候，梯度下降就会更有效率  
但是这里m<n,也就是样本数量相对较少，如果样本数量增多，那么梯度下降又会比较慢